In [ ]:
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline
from quantopian.pipeline.factors import CustomFactor, SimpleMovingAverage, AverageDollarVolume, RollingLinearRegressionOfReturns
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.data import morningstar
from quantopian.pipeline.filters.morningstar import IsPrimaryShare
from quantopian.pipeline.classifiers.morningstar import Sector

import numpy as np
import pandas as pd

from quantopian.pipeline.filters import Q1500US
import quantopian.experimental.optimize as opt

# Constraint Parameters
MAX_GROSS_LEVERAGE = 1.0
NUM_LONG_POSITIONS = 300
NUM_SHORT_POSITIONS = 300

# Here we define the maximum position size that can be held for any
# given stock.
MAX_SHORT_POSITION_SIZE = 2*1.0/(NUM_LONG_POSITIONS + NUM_SHORT_POSITIONS)
MAX_LONG_POSITION_SIZE = 2*1.0/(NUM_LONG_POSITIONS + NUM_SHORT_POSITIONS)
## 0.0033333333333333335

# Risk Exposures
MAX_SECTOR_EXPOSURE = 0.10
MAX_BETA_EXPOSURE = 0.20


In [ ]:
class Reversion(CustomFactor):
    """
    Here we define a basic mean reversion factor using a CustomFactor. We
    take a ratio of the last close price to the average price over the
    last 60 days. A high ratio indicates a high price relative to the mean
    and a low ratio indicates a low price relative to the mean.
    標準的なMean Reversionファクターを作ります、
    過去６０日の終値平均のRatioをとって、（あとでかく）
    """
    inputs = [USEquityPricing.close]
    window_length = 60

    def compute(self, today, assets, out, prices):
        # 過去平均に比べて、今日のプライスはどのくらいのRatioなのか,出してネガティブとして出す
        out[:] = -prices[-1] / np.mean(prices, axis=0)

In [ ]:

def make_pipeline():
    """
    Create and return our pipeline.
    We break this piece of logic out into its own function to make it easier to
    test and modify in isolation.
    
    """

    # Create our mean reversion factor by taking the negative of a momentum factor
    
    reversion = Reversion()

    # Classify all securities by sector so that we can enforce sector neutrality later
    sector = Sector()

    # Screen out non-desirable securities by defining our universe.
    universe = Q1500US()

    # By applying a mask to the rank computations, we remove any stocks that failed
    # to meet our initial criteria **before** computing ranks.  This means that the
    # stock with rank 10.0 is the 10th-lowest stock that was included in the Q1500US.
    # 標準偏差
    # filter のかけかたを mask で行ってしまう
    factor_rank = reversion.rank(mask=universe).zscore()

    # Build Filters representing the top and bottom 150 stocks by our ranking system.
    # We'll use these as our tradeable universe each day.
    longs = factor_rank.top(NUM_LONG_POSITIONS)
    shorts = factor_rank.bottom(NUM_SHORT_POSITIONS)

    # The final output of our pipeline should only include
    # the top/bottom 300 stocks by our criteria
    long_short_screen = (longs | shorts)

    # Define any risk factors that we will want to neutralize
    # We are chiefly interested in market beta as a risk factor so we define it using
    # Bloomberg's beta calculation
    # Ref: https://www.lib.uwo.ca/business/betasbydatabasebloombergdefinitionofbeta.html
    beta = 0.66 * RollingLinearRegressionOfReturns(
        target= symbols("SPY"),
        returns_length=5,
        regression_length=260,
        mask=long_short_screen
    ).beta + 0.33*1.0

    # Create pipeline
    pipe = Pipeline(
        columns={
            'longs': longs,
            'shorts': shorts,
            'factor_rank': factor_rank,
            'reversion': reversion,
            'sector': sector,
            'market_beta': beta
        },
        screen=long_short_screen
    )
    return pipe


In [ ]:
pipe = make_pipeline()
start = '2015-6-30'
end = '2016-6-30'
results = run_pipeline(pipe, start, end, )
